In [1]:
import warnings
warnings.filterwarnings("ignore")

from datetime import datetime, timedelta
from pandas import DataFrame, concat, date_range, ExcelWriter, to_datetime
import os
from numpy import isnan, inf, mean
import time
import json
from Calculator import Calculator

from utils import getSchema, changedType
from BacktestBaseFunc import *


parent = os.path.dirname(os.path.abspath("__file__"))
output_path = os.path.join(parent, "Output", "HundredBreakOut")
if not os.path.isdir(output_path):
    os.makedirs(output_path)

c:\users\kevin\appdata\local\programs\python\python37\lib\site-packages\mpl_finance.py:22: DeprecationWarning: 



    Please use `mplfinance` instead (no hyphen, no underscore).

    To install: `pip install --upgrade mplfinance` 

   For more information, see: https://pypi.org/project/mplfinance/


  category=DeprecationWarning)


In [3]:
bt_list = StockList()

In [7]:
def HundredBreakout(df:DataFrame, **kwargs):
    entry_date = None
    exit_date = None
    entry_price = 0
    exit_price = 0
    max_price = 0
    min_price = inf
    sig = pos = 0
    num_breakout_day = int(kwargs.get('num_breakout_day', 100))
    take_profit = float(kwargs.get('take_profit', .1))
    stop_loss = float(kwargs.get('stop_loss', .1))
    
    result = []
    for i, row in enumerate(df.itertuples()):
        if i < num_breakout_day: continue
        # Check Signal without pos
        if not sig and not pos:
            last_highest_close = df.iloc[i-num_breakout_day:i].Close.max()
#             last_highest_close = df.iloc[-num_breakout_day-i:-i].Close.max()
            if row.Close > last_highest_close:
                sig = -1
        # Entry Market
        elif sig and not pos:
#             if row.Ticker == '1538':print(row)
            if not row.Volume or isnan(row.Open): continue
#             print(row)
            pos, sig = sig, 0
            max_price = min_price = entry_price = row.Open
            entry_date = row.Date
            max_price = max(row.High, max_price)
            min_price = min(row.Low, min_price)
        # Check Signal with pos
        elif not sig and pos:
            max_price = max(row.High, entry_price)
            min_price = min(row.Low, entry_price)
            if (row.High / entry_price - 1 >= take_profit):
                sig = -pos
#                 exit_price = row.High
            if (row.Low / entry_price - 1 <= -stop_loss):
                sig = -pos
#                 exit_price = row.Low
        # Exit Market
        elif sig and pos:
            
            if not row.Volume or isnan(row.Open): continue
#             print(row)
            exit_price = row.Open
            exit_date = row.Date
            res = CreateTradeLog(entry_date, exit_date, entry_price, exit_price, max_price, min_price, pos)
            result.append(res)
            entry_date = None
            exit_date = None
            entry_price = 0
            exit_price = 0
            max_price = 0
            min_price = inf
            sig = pos = 0
        if i == df.shape[0]-1 and pos:
            exit_price = row.Close
            exit_date = row.Date
            res = CreateTradeLog(entry_date, exit_date, entry_price, exit_price, max_price, min_price, pos)
            result.append(res)
            entry_date = None
            exit_date = None
            entry_price = 0
            exit_price = 0
            max_price = 0
            min_price = inf
            sig = pos = 0
#     print()
            
    return result
            
            

In [8]:
results = {}
for ticker_info in bt_list:
    print(f"============= Backtest {ticker_info['Ticker']}=============")
    results[ticker_info['Ticker']] = Backtest(HundredBreakout, ticker_info['Ticker'], bt_period=20)
#     break

============= Backtest 1101=============
============= Backtest 1102=============
============= Backtest 1103=============
============= Backtest 1104=============
============= Backtest 1108=============
============= Backtest 1109=============
============= Backtest 1110=============
============= Backtest 1201=============
============= Backtest 1203=============
============= Backtest 1210=============
============= Backtest 1213=============
============= Backtest 1215=============
============= Backtest 1216=============
============= Backtest 1217=============
============= Backtest 1218=============
============= Backtest 1219=============
============= Backtest 1220=============
============= Backtest 1225=============
============= Backtest 1227=============
============= Backtest 1229=============
============= Backtest 1231=============
============= Backtest 1232=============
============= Backtest 1233=============
============= Backtest 1234=============
============= Ba

============= Backtest 1909=============
============= Backtest 2002=============
============= Backtest 2006=============
============= Backtest 2007=============
============= Backtest 2008=============
============= Backtest 2009=============
============= Backtest 2010=============
============= Backtest 2012=============
============= Backtest 2013=============
============= Backtest 2014=============
============= Backtest 2015=============
============= Backtest 2017=============
============= Backtest 2020=============
============= Backtest 2022=============
============= Backtest 2023=============
============= Backtest 2024=============
============= Backtest 2025=============
============= Backtest 2027=============
============= Backtest 2028=============
============= Backtest 2029=============
============= Backtest 2030=============
============= Backtest 2031=============
============= Backtest 2032=============
============= Backtest 2033=============
============= Ba

============= Backtest 2516=============
============= Backtest 2520=============
============= Backtest 2524=============
============= Backtest 2527=============
============= Backtest 2528=============
============= Backtest 2530=============
============= Backtest 2534=============
============= Backtest 2535=============
============= Backtest 2536=============
============= Backtest 2537=============
============= Backtest 2538=============
============= Backtest 2539=============
============= Backtest 2540=============
============= Backtest 2542=============
============= Backtest 2543=============
============= Backtest 2545=============
============= Backtest 2546=============
============= Backtest 2547=============
============= Backtest 2548=============
============= Backtest 2597=============
============= Backtest 2601=============
============= Backtest 2603=============
============= Backtest 2605=============
============= Backtest 2606=============
============= Ba

============= Backtest 3536=============
============= Backtest 3543=============
============= Backtest 3545=============
============= Backtest 3550=============
============= Backtest 3557=============
============= Backtest 3563=============
============= Backtest 3576=============
============= Backtest 3583=============
============= Backtest 3588=============
============= Backtest 3591=============
============= Backtest 3593=============
============= Backtest 3596=============
============= Backtest 3605=============
============= Backtest 3607=============
============= Backtest 3617=============
============= Backtest 3622=============
============= Backtest 3645=============
============= Backtest 3653=============
============= Backtest 3661=============
============= Backtest 3665=============
============= Backtest 3669=============
============= Backtest 3673=============
============= Backtest 3679=============
============= Backtest 3682=============
============= Ba

============= Backtest 6414=============
============= Backtest 6415=============
============= Backtest 6416=============
============= Backtest 6431=============
============= Backtest 6438=============
============= Backtest 6442=============
============= Backtest 6443=============
============= Backtest 6449=============
============= Backtest 6451=============
============= Backtest 6456=============
============= Backtest 6464=============
============= Backtest 6477=============
============= Backtest 6491=============
============= Backtest 6504=============
============= Backtest 6505=============
============= Backtest 6515=============
============= Backtest 6525=============
============= Backtest 6531=============
============= Backtest 6533=============
============= Backtest 6541=============
============= Backtest 6552=============
============= Backtest 6558=============
============= Backtest 6573=============
============= Backtest 6579=============
============= Ba

ValueError: cannot convert float NaN to integer

# Summary

In [9]:
summary_ = []
bt_day = datetime.today()
bt_day_str = bt_day.strftime("%Y%m%d")
for ticker, result in results.items():
    total_cost = 0
#     total_pnl = 0
    total_net = 0
    trade_num = len(result)
    win_num = 0
    loss_num = 0
    hold_period = []
    for res in result:
        total_net += res['Net']
        total_cost += res['TotalCost']
        win_num += int(res['Net'] > 0)
        loss_num += int(res['Net'] < 0)
        hold_period.append(res['HoldingPeriod'])
    summary_.append({
        '代號':ticker,
        '總成本':total_cost,
        '總損益(淨)':total_net,
        '獲利次數':win_num,
        '損失次數':loss_num,
        '總交易次數':trade_num,
        '勝率%':round(win_num / trade_num*100,2) if trade_num else 0,
        '平均持倉時間(日)':mean(hold_period)
    })
#     if result:
#         DataFrame(result).to_csv(os.path.join(output_path, f'{ticker}_{bt_day_str}.csv'), index=False)
        

In [10]:
sum_df = DataFrame(summary_)#.sort_values(['勝率%','總交易次數'], ascending=False)

In [11]:
sum_df['總損益(淨)'].sum()

-24290492

In [12]:
sum_df.to_csv(os.path.join(output_path, 'Summary.csv'),index=False, encoding='utf-8-sig')

In [13]:
sum_df_prob_sorted = sum_df[sum_df['勝率%'] > 60] # sum_df.sort_values(['勝率%'], ascending=False)
sum_df_trade_num_sorted = sum_df[sum_df.總交易次數 >= 20]#sum_df.sort_values(['總交易次數'], ascending=False)
sum_df_holding_period_sorted = sum_df[sum_df['平均持倉時間(日)'] <= 40]

In [14]:
top_num = 100
prob_Ticker = sum_df_prob_sorted.代號 # .iloc[:top_num]
trade_num_Ticker = sum_df_trade_num_sorted.代號 # .iloc[:top_num]
holding_period_Ticker = sum_df_holding_period_sorted.代號
suitable_Ticker = list(set(prob_Ticker).intersection(trade_num_Ticker).intersection(holding_period_Ticker))

In [15]:
print(sum_df[sum_df.代號.isin(suitable_Ticker)].shape)
sum_df[sum_df.代號.isin(suitable_Ticker)].sort_values("勝率%", ascending=False) # ['總損益(淨)'].sum()

(16, 8)


,代號,總成本,總損益(淨),獲利次數,損失次數,總交易次數,勝率%,平均持倉時間(日)
625,3694,3910,26540,20,8,28,71.43,13.500000
335,2424,5547,27003,29,14,43,67.44,23.302326
181,1762,6775,44325,15,8,23,65.22,17.086957
260,2312,3360,16900,27,15,42,64.29,38.619048
374,2477,7924,47836,35,20,55,63.64,24.345455
754,6155,8744,21856,29,17,46,63.04,31.652174
560,3164,8338,-2838,22,13,35,62.86,24.314286
897,8940,5549,20151,25,15,40,62.50,28.400000
873,8271,5830,25320,18,11,29,62.07,38.344828
83,1456,3979,4611,37,23,60,61.67,13.516667
